In [2]:
import os
import pandas as pd
#import sqlalchemy
import numpy as np

from utils import *

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Lectura de datos del SII

In [3]:
data_path = os.getcwd() + "/Datos/Detalle_Cat_y_Rol_Cobro"

In [4]:
with open(data_path + "/BRTMPNACROL_NAC_2018_1", "rb") as file:
    i=0
    for line in file.readlines():
        print(line)
        if i > 3:
            break
        i+=1

b'0110120181       M LIRA 315                              0000100004N00000000000000000001748932160000001748932162055UO\r\n'
b'0110120181       M LIRA 315                              0000100005N00000000000000000000775787800000000775787802055UO\r\n'
b'0110120181       M LIRA S/N TERMINAL PESQUERO            0000100008N00000027192850000010695362860000000000000000000UC\r\n'
b'0110120181A      M LIRA S/N EXPL TERM PESQUERO           0000100017N00000000416680000000129411160000000000000000000UV\r\n'
b'0110120181       CDTE SAN MARTIN 141 2 DO PISO           0000200004N00000000000000000002704667150000002704667152055UO\r\n'


In [ ]:
%%time
df = pd.DataFrame()
df_dict = {}
limit = 10000000
with open(data_path + "/BRTMPNACROL_NAC_2018_1", "r") as file:
    i=0
    for line in file.readlines():
        line = line#.decode('utf-8')
        data = get_register_from_line_custom(line.replace("\n",""))
        codigo_destino = data[-1]
        
        if codigo_destino == "H":
            df_dict[i] = data
            i+=1
    
        if i >= limit:
            break
        
    

In [ ]:
columns = ["comuna_actual", "año", "semestre", "indicador_de_aseo",
           #"direccion_predial", "manzana_actual", "predio_actual",
           "codigo_de_serie", "cuota_trimestral", "avaluo_total", 
           "avaluo_exento", #"año_termino_exencion", 
           "codigo_ubicacion",
           "codigo_destino"]

df = pd.DataFrame.from_dict(df_dict, orient='index', columns=columns)
df.head()

In [ ]:
corte = 169144585
exento = 47360490
tasa_1 = 0.00893
tasa_2 = 0.01042

df["avaluo_afecto"] = df["avaluo_total"] - df["avaluo_exento"]
df["contribucion_total"] = df["cuota_trimestral"]*2
# caso a: 
df["impuesto_territorial"] = np.where(df["avaluo_total"] <= exento, 0, np.nan)
# caso b:
df["impuesto_territorial"] = np.where((df["avaluo_total"] > exento)&(df["avaluo_total"] <= corte), 
                                      tasa_1*df["avaluo_afecto"]/2,
                                      np.nan)

df["derechos_aseo"] = df["contribucion_total"] - df["impuesto_territorial"]
df[abs(df["derechos_aseo"])<0].round(2)

In [7]:
df["impuesto_territorial"] = np.where(df["avaluo_afecto"] == 0,# caso a
                                      0,
                                      np.where((0 < df["avaluo_afecto"])&(df["avaluo_total"] <= corte), #caso b
                                               tasa_1*(df["avaluo_afecto"])/2,
                                               np.where(df["avaluo_total"] > corte, # caso c
                                                        tasa_1*(corte - df["avaluo_exento"]) + tasa_2*(df["avaluo_total"] - corte)/2,
                                                        np.nan)
                                              )
                                     )

In [8]:
df

,comuna_actual,año,semestre,indicador_de_aseo,codigo_de_serie,cuota_trimestral,avaluo_total,avaluo_exento,codigo_ubicacion,codigo_destino,avaluo_afecto,contribucion_total,impuesto_territorial,derechos_aseo
0,1101,2022,2,,N,0,29900918,29900918,U,H,0,0,0.000000,NaN
1,1101,2022,2,,N,0,26376579,26376579,U,H,0,0,0.000000,NaN
2,1101,2022,2,,N,0,18642448,18642448,U,H,0,0,0.000000,NaN
3,1101,2022,2,,N,0,17804425,17804425,U,H,0,0,0.000000,NaN
4,1101,2022,2,,N,0,21330658,21330658,U,H,0,0,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5664136,16404,2022,2,A,N,24357,61543701,50723085,U,H,10820616,48714,48314.050440,399.949560
5664137,16404,2022,2,A,N,80844,96188659,50723085,U,H,45465574,161688,203003.787910,-41315.787910
5664138,16404,2022,2,A,N,49797,70416544,50723085,U,H,19693459,99594,87931.294435,11662.705565
5664139,16404,2022,2,A,N,24357,66276026,50723085,U,H,15552941,48714,69443.881565,-20729.881565
